In [1]:
!pip install -U ultralytics wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.18.7
    Uninstalling wandb-0.18.7:
      Successfully uninstalled wandb-0.18.7


In [2]:
import random
import shutil
import torch
import yaml
import os

import numpy as np

from pathlib import Path

from ultralytics import YOLO

from sklearn.model_selection import train_test_split

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
shutil.copy2('/content/drive/MyDrive/ITaS/data/best.pt', 'best.pt')

'best.pt'

In [5]:
import wandb
from wandb.integration.ultralytics import add_wandb_callback

wandb.login(key='1de7addec37341330a8aef9bfc7be382cc9c2824')

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sunests. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
def create_yaml(output_yaml_path, train_image_dir, val_image_dir, test_image_dir=' ', nc=10):

    names = ['lungs', 'trachea', 'bronchitis', 'pneumonia/bronchopneumonia', 'pulmonary edema',
             'hydrothorax', 'pneumothorax', 'tracheal collapse', 'neoplasm', 'atelectasis']

    yaml_data = {
        'names': names,
        'nc': nc,
        'train': train_image_dir,
        'val': val_image_dir,
        'test': test_image_dir
    }

    with open(output_yaml_path, 'w') as j:
        yaml.dump(yaml_data, j, default_flow_style=False)

In [7]:
def calculation_uncertainty_sampling_dataset(
    confidence_threshold: float = 0.01
):
    path_to_dataset = '/content/drive/MyDrive/ITaS/data/full_train'

    # train_labels_paths не создаем, так как пройдя через алгоритм Least Confidence
    # мы получим урезанный список картинок, которму train_labels_paths не будет соответствовать.
    # Легче потом просто использовать str.replace(), так как наименование файлов одинаковое.
    train_images_paths = []

    val_images_paths = []
    val_labels_paths = []

    # Проходимся по директории полного датасета
    for root, _, files in os.walk(path_to_dataset):
        for file in files:
            # train часть
            if r'/train/images' in root:
                train_images_paths.append(os.path.join(root, file))
            # val часть
            elif r'/val/images' in root:
                val_images_paths.append(os.path.join(root, file))
            elif r'/val/labels' in root:
                val_labels_paths.append(os.path.join(root, file))

    # Алгоритм Least Confidence
    model = YOLO('best.pt')

    selected_train_images_samples = []
    for i, image_path in enumerate(train_images_paths):
        print(f'{i}/{len(train_images_paths)} ->> {image_path}')
        results = model.predict(image_path, conf=confidence_threshold)
        result = results[0]

        class_probabilities = [[result[[i]].boxes.conf.cpu().numpy()[0] for i, class_number in enumerate(result.boxes.cls) if class_number == class_num] for class_num in set(result.boxes.cls.cpu().numpy())]

        max_probabilities = np.array([np.max(prob) for prob in class_probabilities])
        uncertainty = np.mean(1 - max_probabilities)

        selected_train_images_samples.append({'image_path': image_path, 'uncertainties': uncertainty})

    # Сортируем по приоритету
    selected_train_images_samples.sort(key=lambda x: x['uncertainties'], reverse=True)

    # Вычленяем только пути у сортированного словаря
    selected_train_images_samples = [image['image_path'] for image in selected_train_images_samples]
    selected_train_labels_samples = [path.replace('images', 'labels').replace('.jpg', '.txt') for path in selected_train_images_samples]

    return (selected_train_images_samples, selected_train_labels_samples, val_images_paths, val_labels_paths)

In [8]:
uncertainty_sampling_full_dataset = calculation_uncertainty_sampling_dataset()

WARNING ⚠️ best.pt appears to require 'dill', which is not in Ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.4s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

0/1105 ->> /content/drive/MyDrive/ITaS/data/full_train/train/images/1.1.3.1.40.jpg

image 1/1 /content/drive/MyDrive/ITaS/data/full_train/train/images/1.1.3.1.40.jpg: 640x928 1 lungs, 4 tracheas, 4 bronchitiss, 8 pneumonia/bronchopneumonias, 145.3ms
Speed: 16.4ms preprocess, 145.3ms inference, 890.0ms postprocess per image at shape (1, 3, 6

In [9]:
def create_uncertainty_sampling_dataset(
    train_images_path: list,
    train_labels_path: list,
    val_images_path: list,
    val_labels_path: list,
    experiment_name: str,
    percentage_from_top: int = None,
    number_from_top: int = None,
    alg_name: str = 'uncertainty_sampling'
):
    if percentage_from_top:
        train_images_path = train_images_path[:int(len(train_images_path) * (percentage_from_top / 100))]
        train_labels_path = train_labels_path[:int(len(train_labels_path) * (percentage_from_top / 100))]
    elif number_from_top:
        train_images_path = train_images_path[:number_from_top]
        train_labels_path = train_labels_path[:number_from_top]

    train_images_dir = os.path.join('data', alg_name, experiment_name, 'train', 'images')
    val_images_dir = os.path.join('data', alg_name, experiment_name, 'val', 'images')
    train_labels_dir = os.path.join('data', alg_name, experiment_name, 'train', 'labels')
    val_labels_dir = os.path.join('data', alg_name, experiment_name, 'val', 'labels')

    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(val_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)

    for train_image_path, train_label_path in zip(train_images_path, train_labels_path):
        shutil.copy2(train_image_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'train', 'images',
                          train_image_path[train_image_path.find('images') + 7:]))

        shutil.copy2(train_label_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'train', 'labels',
                          train_label_path[train_label_path.find('labels') + 7:]))

    for val_image_path, val_label_path in zip(val_images_path, val_labels_path):

        shutil.copy2(val_image_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'val', 'images',
                          val_image_path[val_image_path.find('images') + 7:]))

        shutil.copy2(val_label_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'val', 'labels',
                          val_label_path[val_label_path.find('labels') + 7:]))

    yaml_path = os.path.join('data', alg_name, experiment_name, 'data.yaml')
    train_path = os.path.join('train', 'images')
    val_path = os.path.join('val', 'images')
    create_yaml(yaml_path, train_path, val_path)

In [10]:
create_uncertainty_sampling_dataset(
    train_images_path=uncertainty_sampling_full_dataset[0],     # train_images
    train_labels_path=uncertainty_sampling_full_dataset[1],     # train_labels
    val_images_path=uncertainty_sampling_full_dataset[2],       # val_images
    val_labels_path=uncertainty_sampling_full_dataset[3],       # val_samples
    percentage_from_top=1,
    experiment_name='1_train'
)

In [11]:
create_uncertainty_sampling_dataset(
    train_images_path=uncertainty_sampling_full_dataset[0],     # train_images
    train_labels_path=uncertainty_sampling_full_dataset[1],     # train_labels
    val_images_path=uncertainty_sampling_full_dataset[2],       # val_images
    val_labels_path=uncertainty_sampling_full_dataset[3],       # val_samples
    percentage_from_top=10,
    experiment_name='10_train'
)

In [12]:
create_uncertainty_sampling_dataset(
    train_images_path=uncertainty_sampling_full_dataset[0],     # train_images
    train_labels_path=uncertainty_sampling_full_dataset[1],     # train_labels
    val_images_path=uncertainty_sampling_full_dataset[2],       # val_images
    val_labels_path=uncertainty_sampling_full_dataset[3],       # val_samples
    percentage_from_top=20,
    experiment_name='20_train'
)

In [13]:
def train_yolo_model(main_experiment: str, name: str, seed: int):
    wandb.init(project='ITaS', job_type='training')

    model = YOLO("yolov8m-seg.pt")

    results = model.train(
        data = os.path.join('/content/data', main_experiment, name, 'data.yaml'),
        project = 'ITaS',
        name = name,
        epochs = 25,
        patience = 0,
        batch = 5,
        imgsz = 640,
        seed=seed
    )
    wandb.finish()

In [14]:
for i in range(5):
    train_yolo_model(main_experiment='uncertainty_sampling', name=f'1_train', seed=(i + 1) * 13)

100%|██████████| 52.4M/52.4M [00:00<00:00, 113MB/s]


Ultralytics 8.3.50 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/content/data/uncertainty_sampling/1_train/data.yaml, epochs=25, time=None, patience=0, batch=5, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=ITaS, name=1_train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=13, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

100%|██████████| 755k/755k [00:00<00:00, 23.6MB/s]


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 96.9MB/s]


...

       6/25      3.09G      1.935      5.295      3.737      2.306          3        640: 100%|██████████| 3/3 [00:00<00:00,  3.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:08<00:00,  3.19it/s]

                   all        277       1402      0.964     0.0446     0.0544      0.028      0.942     0.0308      0.027    0.00888



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       7/25      3.07G      1.816      4.955      3.521      2.113          6        640: 100%|██████████| 3/3 [00:00<00:00,  3.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  32%|███▏      | 9/28 [00:03<00:06,  2.90it/s]


KeyboardInterrupt: 

In [ ]:
for i in range(5):
    train_yolo_model(main_experiment='uncertainty_sampling', name=f'10_train', seed=(i + 1) * 14)

In [ ]:
for i in range(5):
    train_yolo_model(main_experiment='uncertainty_sampling', name=f'20_train', seed=(i + 1) * 15)